In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('./data/train.csv')
data.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [3]:
features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'Geography', 'Gender', 'HasCrCard', 'IsActiveMember']
x_data = pd.get_dummies(data[features]).astype(np.float32)
y_data = (data['Exited']).astype(np.float32)

scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)

x_train, x_, y_train, y_ = train_test_split(x_data, y_data, test_size=0.33, random_state=1)

In [27]:
def evaluate(model):
    predictions = model.predict(x_)
    predictions = predictions.reshape(1, len(predictions))[0]
    predictions = np.array(list(map(lambda p: 1 if p >= 0.5 else 0, predictions)))

    miss_rate = round(np.mean(predictions !=  y_) * 100 , 2)
    print(f'Miss rate: {miss_rate}%')

    return miss_rate

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [31]:
model_nn = Sequential(
    [
        Dense(256, activation = 'relu'),
        Dense(128, activation = 'relu'),
        Dense(1, activation = 'sigmoid'),
    ]
)

model_nn.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
)

model_nn.fit(x_train, y_train, epochs=10)

Epoch 1/10
3456/3456 [==============================] - 5s 1ms/step - loss: 0.3387
Epoch 2/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3275
Epoch 3/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3254
Epoch 4/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3243
Epoch 5/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3235
Epoch 6/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3225
Epoch 7/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3219
Epoch 8/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3212
Epoch 9/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3207
Epoch 10/10
3456/3456 [==============================] - 4s 1ms/step - loss: 0.3204


In [32]:
from sklearn.linear_model import LogisticRegression, SGDRegressor

model_lr = LogisticRegression()
model_lr.fit(x_train, y_train)

model_sgdr = SGDRegressor()
model_sgdr.fit(x_train, y_train)

SGDRegressor()

In [33]:
models = [model_nn, model_lr, model_sgdr]
miss_rates = []

for model in models:
    miss_rates.append(evaluate(model))

1702/1702 [==============================] - 1s 723us/step
Miss rate: 13.74%
Miss rate: 16.56%
Miss rate: 17.27%


In [25]:
best_model = models[np.argmin(miss_rates)]

In [26]:
test_data = pd.read_csv("./data/test.csv")
x_test = pd.get_dummies(test_data[features]).astype('float32')

x_test = scaler.fit_transform(x_test)

test_pred = best_model.predict(x_test)
[test_pred] = test_pred.reshape(1, len(test_pred))
test_pred = np.round(test_pred, 1)
output = pd.DataFrame({'id': test_data.id, 'Exited': test_pred})
output.to_csv('./data/bank-churn-submission.csv', index=False)

3439/3439 [==============================] - 2s 719us/step
